<a href="https://colab.research.google.com/github/ArseniyKoz/uni.neuralnetworks/blob/main/ltsm_Dostoevskiy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
with open('dostoevsky.txt', 'r') as f:
  text = f.read()
text[:1000]

'Федор Михайлович Достоевский\nБедные люди\nОх уж эти мне сказочники! Нет чтобы написать что-нибудь полезное, приятное, усладительное, а то всю подноготную в земле вырывают!.. Вот уж запретил бы им писать! Ну, на что это похоже: читаешь… невольно задумываешься, — а там всякая дребедень и пойдет в голову; право бы, запретил им писать; так-таки просто вовсе бы запретил.\nКн. В. Ф. Одоевский\nАпреля 8-го\nБесценная моя Варвара Алексеевна!\nВчера я был счастлив, чрезмерно счастлив, донельзя счастлив! Вы хоть раз в жизни, упрямица, меня послушались. Вечером, часов в восемь, просыпаюсь (вы знаете, маточка, что я часочек-другой люблю поспать после должности), свечку достал, приготовляю бумаги, чиню перо, вдруг, невзначай, подымаю глаза, — право, у меня сердце вот так и запрыгало! Так вы-таки поняли, чего мне хотелось, чего сердчишку моему хотелось! Вижу, уголочек занавески у окна вашего загнут и прицеплен к горшку с бальзамином, точнехонько так, как я вам тогда намекал; тут же показалось мне,

In [ ]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

In [ ]:
encoded = np.array([char2int[ch] for ch in text])
encoded[:100]

array([121,  17,  36,  67,  26,  59,  24,  15,   0,   1,  64,  84,  67,
        20,  15, 100,  59,  55,  67,   9,  86,  67,  17,  20,   9,  27,
        15,  64,  89,  88,  17,  36,  92,  49,  17,  59,  84, 129,  36,
        15,  89,  29,   0,  59,  95,  93,  59,   7,  86,  15,  59, 115,
        92,  17,  59,   9,  27,   1, 114,  67, 100,  92,  15,  27,  15,
        74,  59, 108,  17,  86,  59, 100,  86,  67, 126,  49,  59,  92,
         1,  83,  15,   9,   1,  86, 131,  59, 100,  86,  67, 122,  92,
        15, 126,  95,  36, 131,  59,  83,  67,  84])

In [ ]:
def one_hot_encode(arr, n_labels):

  one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
  one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
  one_hot = one_hot.reshape((*arr.shape, n_labels))

  return one_hot

In [ ]:
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)
print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [ ]:
def get_batches(arr, batch_size, seq_length):
  '''Create a generator that returns batches of size
  batch_size x seq_length from arr.

  Arguments
  ---------
  arr: Array you want to make batches from
  batch_size: Batch size, the number of sequences per batch
  seq_length: Number of encoded chars in a sequence
  '''

  ## TODO: Get the number of batches we can make
  n_batches = len(arr) // (batch_size*seq_length)

  ## TODO: Keep only enough characters to make full batches
  arr = arr[:batch_size*n_batches*seq_length]

  ## TODO: Reshape into batch_size rows
  arr = arr.reshape(batch_size, -1)

  ## Iterate over the batches using a window of size seq_length
  for n in range(0, arr.shape[1], seq_length):
    # The features
    x = arr[:, n:n+seq_length]
    # The targets, shifted by one
    y = np.zeros_like(x)
    try:
      y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
    except IndexError:
      y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
    yield x, y

In [ ]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

print('x\n', x)
print('\ny\n', y[:10, :10])

x
 [[121  17  36  67  26  59  24  15   0   1  64  84  67  20  15 100  59  55
   67   9  86  67  17  20   9  27  15  64  89  88  17  36  92  49  17  59
   84 129  36  15  89  29   0  59  95  93  59   7  86  15]
 [ 84  10  59  95  84  49 126   1  43   9 131  59 100  17  26  17 114  59
    9  15  84  95  10  59 100  86  67 126  49  59  95  36  17  26  93   1
   86 131  59  20  59  36  95  34  17  59  92   1  27  15]
 [ 59  20  59  83  26  15  20  49 100  92  67 115  59  95  98  84  17  59
   92   1  36  67  84  98  67  11  59 110   1  27 122  86  67  59  84  95
  100  34  17  59  86   1 115  10  59  98  36  17  59  83]
 [  9 131  59  67   9  86   1  20   1  86 131   9  43  59  86  17  83  17
   26 131  11  59 109   9  84  15  59 126  49  59  92  17  59  98  26  95
    9  86  92  67  59 126  49  84  67  10  59  43  59 126]
 [ 36 131  59  92   1 115  17  26  17  92  15  17 115  59  67  92  15  59
   92  17  59 115  67  98  84  15  59   7  86  67  98  67  59   9  36  17
   84   1  86 131  11

In [ ]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
  print('Training on GPU!')
else:
  print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [ ]:
class CharRNN(nn.Module):
  def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.3  , lr=0.001):
    super().__init__()
    self.drop_prob = drop_prob
    self.n_layers = n_layers
    self.n_hidden = n_hidden
    self.lr = lr

    # creating character dictionaries
    self.chars = tokens
    self.int2char = dict(enumerate(self.chars))
    self.char2int = {ch: ii for ii, ch in self.int2char.items()}
    self.vocab_size = len(char2int)

    ## TODO: define the layers of the model
    self.dropout = nn.Dropout(self.drop_prob)

    self.lstm = torch.nn.LSTM(self.vocab_size, self.n_hidden, num_layers=self.n_layers, dropout=self.drop_prob, batch_first=True)
    #self.lstm = torch.nn.LSTM(self.vocab_size, hidden_size = self.n_hidden, num_layers=self.n_layers, dropout=self.drop_prob, bidirectional=True, batch_first=True)
    self.fc = nn.Linear(self.n_hidden, self.vocab_size) #<linear>
    #self.fc = nn.Linear(2*self.n_hidden, self.vocab_size)

  def forward(self, x, hidden):
    ''' Forward pass through the network.
    These inputs are x, and the hidden/cell state `hidden`. '''

    ## TODO: Get the outputs and the new hidden state from the lstm
    #print(hidden)
    r_output, hidden = self.lstm(x, hidden)


    ## TODO: pass through a dropout layer
    out = self.dropout(r_output)

    # Stack up LSTM outputs using view
    # you may need to use contiguous to reshape the output
    out = out.contiguous().view(-1,self.n_hidden)

    ## TODO: put x through the fully-connected layer
    out = self.fc(out)


    # return the final output and the hidden state
    return out, hidden


  def init_hidden(self, batch_size):
    ''' Initializes hidden state '''
    # Create two new tensors with sizes n_layers x batch_size x n_hidden,
    # initialized to zero, for hidden state and cell state of LSTM
    weight = next(self.parameters()).data

    if (train_on_gpu):
      hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
    else:
      hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())

    return hidden

In [ ]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network
        Arguments
        ---------
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    '''
    net.train()
    opt = torch.optim.Adam(net.parameters(), lr=lr, amsgrad=True)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
      opt,
      patience=5,
      verbose=True,
      factor=0.5
    )
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if(train_on_gpu):
      net.cuda()

    loss_avg = []

    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
      #net.train()
    # initialize hidden state
      h = net.init_hidden(batch_size)

      for x, y in get_batches(data, batch_size, seq_length):
        #print(x, y)
        counter += 1

        # One-hot encode our data and make them Torch tensors
        x = one_hot_encode(x, n_chars)
        inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

        if(train_on_gpu):
          inputs, targets = inputs.cuda(), targets.cuda()
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])
        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        output, h = net(inputs, h)

        # calculate the loss and perform backprop
        loss = criterion(output, targets.view(batch_size*seq_length).long())
        loss.backward()

        loss_avg.append(loss.item())

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        opt.step()

        # loss stats
        if counter % print_every == 0:
          # Get validation loss
          val_h = net.init_hidden(batch_size)
          val_losses = []

          mean_loss = np.mean(loss_avg)
          scheduler.step(mean_loss)
          loss_avg = []

          net.eval()
          for x, y in get_batches(val_data, batch_size, seq_length):
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            x, y = torch.from_numpy(x), torch.from_numpy(y)

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            val_h = tuple([each.data for each in val_h])

            inputs, targets = x, y
            if(train_on_gpu):
              inputs, targets = inputs.cuda(), targets.cuda()
            output, val_h = net(inputs, val_h)
            val_loss = criterion(output, targets.view(batch_size*seq_length).long())

            val_losses.append(val_loss.item())

          net.train() # reset to train mode after iterationg through validation data

          print("Epoch: {}/{}...".format(e+1, epochs),
                "Step: {}...".format(counter),
                "Loss: {:.4f}...".format(loss.item()),
                "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [ ]:
## TODO: set your model hyperparameters
# define and print the net
n_hidden = 128
n_layers = 2
dropout = 0.2
net = CharRNN(chars, n_hidden, n_layers, dropout)
print(net)
batch_size = 8
seq_length = 128
n_epochs = 32 # start small if you are just testing initial behavior
# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.01, print_every=1000)

CharRNN(
  (dropout): Dropout(p=0.2, inplace=False)
  (lstm): LSTM(133, 128, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=133, bias=True)
)
Epoch: 3/32... Step: 1000... Loss: 1.8191... Val Loss: 2.0977
Epoch: 6/32... Step: 2000... Loss: 1.8368... Val Loss: 1.9204
Epoch: 8/32... Step: 3000... Loss: 1.6389... Val Loss: 1.8572
Epoch: 11/32... Step: 4000... Loss: 1.6196... Val Loss: 1.8422
Epoch: 13/32... Step: 5000... Loss: 1.5582... Val Loss: 1.8061
Epoch: 16/32... Step: 6000... Loss: 1.5135... Val Loss: 1.8074
Epoch: 19/32... Step: 7000... Loss: 1.6806... Val Loss: 1.7944
Epoch: 21/32... Step: 8000... Loss: 1.7507... Val Loss: 1.7785
Epoch: 24/32... Step: 9000... Loss: 1.5056... Val Loss: 1.7887
Epoch: 26/32... Step: 10000... Loss: 1.6463... Val Loss: 1.7759
Epoch: 29/32... Step: 11000... Loss: 1.5612... Val Loss: 1.7819
Epoch: 32/32... Step: 12000... Loss: 1.5941... Val Loss: 1.7739


In [ ]:
model_name = 'rnn_x_epoch.net'
checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}
with open(model_name, 'wb') as f:
  torch.save(checkpoint, f)

In [ ]:
def predict(net, char, h=None, top_k=None):
  ''' Given a character, predict the next character.
  Returns the predicted character and the hidden state.
  '''

  # tensor inputs
  x = np.array([[net.char2int[char]]])
  x = one_hot_encode(x, len(net.chars))
  inputs = torch.from_numpy(x)

  if(train_on_gpu):
    inputs = inputs.cuda()

  # detach hidden state from history
  h = tuple([each.data for each in h])
  # get the output of the model
  out, h = net(inputs, h)
  # get the character probabilities
  p = F.softmax(out, dim=1).data
  if(train_on_gpu):
    p = p.cpu() # move to cpu

  # get top characters
  if top_k is None:
    top_ch = np.arange(len(net.chars))
  else:
    p, top_ch = p.topk(top_k)
    top_ch = top_ch.numpy().squeeze()

  # select the likely next character with some element of randomness
  p = p.numpy().squeeze()
  char = np.random.choice(top_ch, p=p/p.sum())

  # return the encoded value of the predicted char and the hidden state
  return net.int2char[char], h

In [ ]:
def sample(net, size, prime='The', top_k=None):
  if(train_on_gpu):
    net.cuda()
  else:
    net.cpu()

  net.eval() # eval mode

  # First off, run through the prime characters
  chars = [ch for ch in prime]
  h = net.init_hidden(1)
  for ch in prime:
    char, h = predict(net, ch, h, top_k=top_k)
  chars.append(char)

  # Now pass in the previous character and get a new one
  for ii in range(size):
    char, h = predict(net, chars[-1], h, top_k=top_k)
    chars.append(char)
  return ''.join(chars)
print(sample(net, 1000, prime='Анна', top_k=5))

Анна нет их и не будет и нужно с самое себя положить; но он на какая-то из сердце меня оставления, не скажу с первую подразумитель об деньги с сада, что я стало быть, будет, что ваша, крестью, и столько стал сердечко с вас стоял оставают, как бы вы меня нет от материя мысль. В такого... Но вы с вами, что он не мог не больчик вы по перед беспререссивание своим моем молоды, кресть в долго просто, и собразоваться несчастие, и все пилать. Ведь вот чувством собора, что они об карту стало бы все это наделная и по потила, как-то благодетерный матушку своих добра, но в послишать предлагать веселость. Но он не просил из собрание. Я сам мне было нужно вас бы не более на садинко по своему.
Ваша
Вечера нет, Настенька, потому что он божий десять странная строхольше, но вот одна стало божий мой, чтобы он, которая была были последние моим и сегорости на нашего денег с ним во сказать, и только не была на всю меня ни знаю; всю слова, краснет, на своей, который посла остальна на нас прядная володенется,

In [ ]:
with open('rnn_x_epoch.net', 'rb') as f:
  checkpoint = torch.load(f)

loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])
# Sample using a loaded model
print(sample(loaded, 1000, prime="И он сказал ",  top_k=5))

<ipython-input-17-ac01ad47a0f9>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f)


И он сказал на него доме, не было. Прошелт с душеть, потому что я полный мы вызатернуться, чиновник, и пословиче. Он достойную. Я-то ведь никогда не можете, что он вот вам одиночита письмо неприятно и только всё сладекое посмеле, как ни вымутет, не могу; ну, на нем один печать и ваши постушом. Посмотрил, что вас, что я под сердце! Обо всех после друга моим думало, что я вам совершенно больше на месте полнул из подобности платка мою и потолки, по просту не молоду, что мне сердиться. Я большая, когда вы не стало быть, в сегодня, кончивый последную мечтать и с нимом, на подрожало себь! Она отказывая моим вас, и нужно стало ужасную в самом деле он про наши на восмоверенно со мною странного держал и не стало быть, он боялся, что-нибудь, чтоб все на нам слезы наделанность ни наприличению про малицым верите, чем был привидений нось не могла не сказали, что бы было надобно и блина, подумом, послушайте, чтобы она одна, к совершенно, как будто был простино, потому что вот, не быть, и пример, и в